# Hyperparameter Tuning using HyperDrive

Import Dependencies.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice

import os
import shutil
import joblib

### Load Workspace

Load the exisiting azure workspace. 

In [ ]:
ws = Workspace.from_config()
experiment_name = 'loan_default_model'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

### Compute Cluster Configuration

Setting up the virtual machine and other cluster configuration needed in running the experiment. 

In [ ]:
cpu_cluster_name = "aml-cluster" 

#verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print(f"Found existing cluster: {cpu_cluster_name} to be used.")
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2', max_nodes=6)

    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

## Hyperdrive Configuration

The project entails identifying customers who would either default or not after taking a loan credit. This transalates to a binary classification: to default or not default. Hence, a Logistic regression model would be used. SKLearn's logistic regression algorithm is a well-known supervised learning approach optimized for dichotomous or binary variables.

Hyperparameters are adjustable parameters that let you control the model training process. Hyperparameter tuning is the process of finding the configuration of hyperparameters that results in the best performance. The process is typically computationally expensive and manual.
The two hyperparamters used in this experiment are `C` and `max_iter`. `C` is the Inverse Regularization Strength which applies a penalty to stop increasing the magnitude of parameter values in order to reduce overfitting. `max_iter` is the maximum iteration to converge for the SKLearn Logistic Regression algorithm.

Primary Metric is used to optimize the hyperparamter tuning. Each training run is evaluated for the primary metric.
We have chosen `Accuracy` as the primary metric. `MAXIMIZE` is the preferred primary metric goal.

Early termination policies are applied to HyperDrive runs. A run is cancelled when the criteria of a specified policy are met. The `BanditPolicy` was used.
It is based on slack factor/slack amount and evaluation interval. Bandit terminate runs where the primary metric is not within the specified slack factor/slack amount compared to the best performing run. This helps to improves computational efficiency.

For this experiment, `evaluation_interval=1`, `slack_factor=0.2`, and `delay_evaluation=4`. This configration means that the policy would be applied to every even number iteration of the pipeline greater than 4 and if 1.2*value of the benchmark metric for current iteration is smaller than the best metric value so far, the run will be cancelled.

In [ ]:
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=4)

param_sampling = RandomParameterSampling(parameter_space={"--C": choice(1, 2, 3, 4, 5, 6), "--max_iter": choice(50, 100, 150, 200, 250, 300)})

# Code below creates a new training directory with the train.py script in it
if "training" not in os.listdir():
    os.mkdir("./training")

shutil.copy('train.py', "./training")

# Code creates an output directory to store trained models
if "outputs" not in os.listdir():
    os.makedirs('./outputs', exist_ok=True)

estimator =  SKLearn(source_directory='./training', entry_script="train.py", compute_target=cpu_cluster)

hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                                     hyperparameter_sampling=param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

### Submitting the experiment

In [ ]:
print('Submitting Hyperdrive experiment...')
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

Use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

Get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Details:', best_run,
      'Best Run ID:', best_run.id, 
      'Arguments:', best_run.get_details()['runDefinition']['arguments'],
      'Accuracy:', best_run_metrics['Accuracy'],
      'Regularization Strength (C): ', best_run_metrics['Regularization Strength:'],
      'Maximum Iterations (max_iter): ', best_run_metrics['Max iterations:'], sep = '\n')

In [ ]:
# Save the best model
best_model = best_run.register_model(
    model_name='HypDriveBestModel', 
    model_path='outputs/', 
    properties={'Accuracy': best_run_metrics['Accuracy'], 
                'Regularization Strength (C)': best_run_metrics['Regularization Strength:'], 
                'Maximum Iterations (max_iter)': best_run_metrics['Max iterations:']})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service